# Homework 3

### This is the solution of:
* Student1
* Student2
* Student3


The same general rules as for Homework 1 applies.

$$\newcommand{\dx}{\,\mathrm{d}x}
\newcommand{\ldb}{\left\llbracket}
\newcommand{\rdb}{\right\rrbracket}
\newcommand{\lp}{\left(}
\newcommand{\rp}{\right)}
\newcommand{\tn}{|\mspace{-1mu}|\mspace{-1mu}|}
\newcommand{\IR}{\mathbb{R}}
$$

## Problem 1 ($L^2$-projection)


Let $0 = x_0 < x_1 < x_2 < \ldots < x_{N} = 1$ be a partition of the interval
$I = (0,1)$ into $N$ subintervals of equal length $h=1/N$.  Moreover,
let $\{ \lambda_j\}_{j=0}^{N}$ be the set of nodal/hat basis functions of $V_h$
associated with the $N+1$ nodes $x_j$, $j = 0,1\ldots, N$, such that
The $L^2$-projection reads: find $P_hf\in V_h$ such that
$$
    \int_I(f - P_h f) v \dx = 0 \qquad\text{for all }v\in V_h
$$
We consider three different functions
* $f_1(x)=x\sin (3\pi x)$
* $f_2(x)=2-10x$
* $f_3(x)=x(1-x)$

**a)** Write a script ${\texttt {AssembleMassMatrix1D(x)}}$ that computes the mass matrix. 

**b)** Write a corresponding function $\texttt{AssembleLoadVector(x,f)}$ which computes 
the load vector for a given function $f$. Make sure that you can easily switch (for example by adding another input variable) out the quadrature
rule (Trapezoid and Simpson) to approximate the integrals $\int_{I} f \varphi_i \dx$

Convince yourself that your code is correct by computing the $L^2$ projection of the given functions, and observe that they look similar to the original function when plotting in the same figure (with enough mesh points). You do not need to document this.

**c)** Write a function $\texttt{ComputeErrorL2(x,Pf,f)}$ which computes the error in $L^2$ norm, 
$$\|P_h f-f \|=\sqrt{\int_0^1(P_h f-f)^2 dx}.$$
Hint: use the equivalent formula
$$\|P_h f-f \|=\sqrt{\sum_{i=1}^{N} \int_{x_{i-1}}^{x_{i}}(P_h f-f)^2 dx},$$
and approximate the integrals in each subinterval using Simpson's rule.

**d)** Test your code by computing the $L^2$-projection $P_h f\in V_h$ for the three given functions by using the
Trapezoidal rule to compute the load vector. Test with different $N$ values. Do the errors converge as expected? Why?

**e)** Repeat **d)** using Simpson's rule to compute the load vector. What do you find and why?

### Code Snippets

In [ ]:
def AssembleMassMatrix1D(x):
    # Number of intervals
    N = x.size-1
    # 1) Allocate and initiate matrix
    M = ...

    # 2) Compute volume contributions by iterating over 
    #    intervals I_0 to I_(N-1):
    for i in range(0,N):
        # Mesh  size
        h = ...
        # Compute local mass matrix
        M_loc = ...*np.array([[2, 1],[1, 2]])
        # Write local matrix into global
        M[i, i] += M_loc[0, 0]
        # Add three more matrix entries
        ...
        ...
        ...
    
    return M

In [ ]:
def AssembleLoadVector1D(x, f):
    # Number of intervals
    N = ...
    # 1) Allocate and initiate global load vector
    b = ...
    # 2) Compute volume contributions by iterating over intervals 
    #    I_1 to I_N:
    for i in range(1,N+1):
        # Mesh  size
        h = ...
        # Element load vector
        b_loc = np.zeros(2)
        # Apply quadrature rule to int f phi_{i-1} and int f phi_{i}
        # Trapezoidal
        b_loc[0] = f(x[i-1])*h/2
        b_loc[1] = ...
        # Simpson
        # m = ...
        # b_loc[0] = ...
        # b_loc[1] = ...
        
        # Add to global vector
        b[i-1] += ...
        b[i] += ...

    return b

In [ ]:
def ComputeErrorL2(x,Pf,f):
    # Pf is an arracy consisting of the L2 projection evaluated on the mesh points
    # f is a function for f1, f2 or f3
    N = Pf.size-1
    err = 0
    for i in range(N):
        h = x[i+1]-x[i]
        Pfl = ... # L2 projection value at x[i]
        Pfr = ... # L2 projection value at x[i+1]
        fl = ... # function value at x[i]
        fr = ... # function value at x[i+1]
        m = (x[i]+x[i+1])/2
        err = err+ ... # Simpsons rule
        #print(err)
    return ... # hint: ... is not just err

## Problem 2  (The variational/weak formulation of Poisson's equation)

Note: Problem 2 is a theoretical exercise. You are not required to write any programming code.

**a)**
Derive the weak formulation for the steady-state advection-diffusion equation with variable coefficents, 
\begin{align*}
- (a(x)u(x)')' + c(x)u(x) = f \quad \text{in } I = (0,1)
\\
u(0) = 0, \quad u'(1) = 10,
\end{align*}
for two smooth functions $a(x)$ and $c(x)$ satisfying $a(x) \geqslant a_0 > 0$
and $c(x) \geqslant 0$.


**b)**
Derive a finite element method in suitable function spaces. 




**c)**
Derive the linear system for the above finite element method. 

__ANSWERS:__

__a)__

To find the weak formulation, we start by multiplying both sides with our test function $v$ and integrating all terms over $I$:

$$- \int_I (a \cdot u')'v \ dx + \int_I c \cdot u \cdot v \ dx = \int_I fv \ dx.$$
Next, we expand the first term on the LHS using integration-by-parts:

 $$-(a(1)u'(1)v(1)-a(0)u'(0)v(0)) + \int_I au'v' \ dx + \int_I c u v \ dx = \int_I fv \ dx$$

Now, we want to define our test space in which we will find our test function $v$. since we want our test function to be zero at the boundary $x = 1$ (to match the Dirichlet boundary condition) and to be well-behaved, we define our test space as: 
$$V_0 = \{ v \ : \ \|v\|_{L^2(I)} + \|v'\|_{L^2(I)} < \infty, v(0) = 0 \}.$$

So, since we require our test function to belong to this space, we consequently require $v(0) = 0$, meaning we can remove the first term on the LHS. Also using the Neumann BC $u'(1) = 10$, will result in the following weak formulation after some rearranging:

find $u(x) \in V $ such that 

$$ \int_I a(x)u'(x)v'(x) \ dx + \int_I c(x) u(x) v(x) \ dx = \int_I fv(x) \ dx + 10a(1)v(1)$$
$$ u(0) = 0, u'(1) = 10$$

where $v \in V_0 = V = \{ v \ : \ \|v\|_{L^2(I)} + \|v'\|_{L^2(I)} < \infty, v(0) = 0 \}, \ \  a(x) \geqslant a_0 > 0
\ $ and $ \ c(x) \geqslant 0$

__b)__

We divide the interval $I$ into $N$ intervals with $N+1$ total grid points, giving us the intervals $I_i, i = 1, \ 2, ... \ , N$

We now define the function $u_h(x) \approx u(x)$ which is continuous and piecewise linear. We also define, from the space of continuous piecewise linear functions on $I_i$:  
$$V_h = \{ v \ : \ v \in C^0(I), \ v|_{I_i} \ \in \  P_1(I_i) \  \text{with i} \  = 1, \ 2, \ ... \ , N\}$$ 

the space in which we want to find our solution $u_h$:

$$u_h \in V_{h,0} = \{ v \in V_h \ : \ v(0) = 0\} $$

Thus, our finite element method formulation is:

Find $u_h \in V_{h,0}$ such that

$$ \int_I a(x)u_h'(x)v_h'(x) \ dx + \int_I c(x) u_h(x) v_h(x) \ dx = \int_I fv_h(x) \ dx + 10a(1)v_h(1), \ \ v \in V_{h,0}$$



__c)__

Now, we declare that we can set our $v_h$ to be our nodal basis functions $\phi_i(x_j)$
since $\phi_i(x_j) \in V_{h,0}, \ $ where 
\begin{align}
  \varphi_i(x_j) =
  \left \{ 
  \begin{array}{l}
    1, \quad \mbox{if } i = j, \\
    0, \quad \mbox{if } i \neq j.
  \end{array}
  \right .
\end{align}

Furthermore, since $u_h \in V_{h,0}$ as well, $u_h$ can be written as a linear combination of the nodal basis functions:

$$u_h(x) = \sum_{j=1}^{N}\xi_j \phi_j(x),$$ 

which also means $$u_h'(x) = \sum_{j=1}^{N}\xi_j \phi_j'(x),$$

where $\xi_j$ are the weights, which are to be determined. Note, we do not include $\phi_0$ because of our Dirichlet BC. 

From this, our FEM formulation becomes: 

$$a(x)\sum_{j=1}^{N}\int_I \xi_j \phi_j' \phi_i' \ dx + c(x)\sum_{j=1}^{N}\int_I \xi_j \phi_j \phi_i \ dx = \int_I f \phi_i \ dx + 10a(1) \phi_1, \ \ \ i,j = 1,2, \cdots , N $$

So, we see that we have our stiffness and mass matrix on the LHS, each multiplied with our functions $a(x)$ and $c(x)$ respectively. In matrix form, this system looks like:

$$a(x) \begin{bmatrix}
\int_I \phi'_1 \phi'_1 & \cdots & \int_I \phi'_1 \phi'_N \\
\int_I \phi'_2 \phi'_1 & \cdots & \int_I \phi'_2 \phi'_N \\
\vdots & \ddots & \vdots \\
\int_I \phi'_N \phi'_1 & \cdots & \int_I \phi'_N \phi'_N
\end{bmatrix}
\begin{bmatrix}
\xi_1 \\
\xi_2 \\
\vdots \\
\xi_N
\end{bmatrix} 
+ 
c(x) \begin{bmatrix}
\int_I \phi_1 \phi_1 & \cdots & \int_I \phi_1 \phi_N \\
\int_I \phi_2 \phi_1 & \cdots & \int_I \phi_2 \phi_N \\
\vdots & \ddots & \vdots \\
\int_I \phi_N \phi_1 & \cdots & \int_I \phi_N \phi_N
\end{bmatrix}
\begin{bmatrix}
\xi_1 \\
\xi_2 \\
\vdots \\
\xi_N
\end{bmatrix} = \begin{bmatrix}
\int_I f \phi_1 \\
\int_I f \phi_2 \\
\vdots \\
\int_I f \phi_N
\end{bmatrix} + + 10a(1) \phi_1$$

## Problem 3 (A 1D Finite Element Solver)

**a**) For $u(x) = x + \cos(2\pi x)$, compute a right-hand side $f$ and boundary values $g_R$, $g_N$
such that $u(x)$ solves the two-point boundary value problem on $I = [0,1]$ given by
\begin{gather}
-u''= f, \quad 0<x<1, 
\\
u'(0)=-g_N, \quad u'(1)= (g_R - u(1))
\end{gather}
a Neumann boundary condition on the left end point and a Robin boundary condition on the right end point.

**b)** Write down the weak form and then the finite element method, and be precise with the function spaces. Implement a finite element solver for this two-point boundary problem. 
Use uniform meshes with $h=1/N$ for $N = 4, 8, 16, 32, 64, 128$ and
compare the numerical solution $u_h$ with the exact solution $u(x)$
by plotting $u_h$ for $N = 4, 8, 16, 128$ and $u$ into the same figure.

**c)** Define the energy norm $\tn v \tn$ for $v \in V$ for this problem. Assuming that you can integrate the
right hand side $\int_I f v_h$ **exactly** for $v \in V_h \subset V$, show that the energy error satisfies
\begin{align*} 
 \tn u-u_h\tn^2=\tn u\tn^2-\tn u_h\tn^2     \quad\quad\quad\quad\quad\quad(1)
\end{align*}
by exploiting the Galerkin orthogonality. Hint: there is a similar identity in the lecture notes. You may follow the same procedure for the proof, but note that the energy norm here has a different expression.

Next, show that the following identity holds: 
\begin{align*}
\tn u_h\tn^2 =  U^T A U              \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad (2)
\end{align*}
where $U$ is the  coefficient vector corresponding to $u_h = \sum_{i=0}^{N} U_i \varphi_i$ and $A$ is the stiffness matrix for the problem at hand (including the contribution from the boundary).

**d)** For the given $N$ in b), compute the errors in the follwoing norms

* a) The maximum norm 
* b) The energy norm  

present the results in log-log plots (error versus $h$) and specify the convergence rate. Do they converge as expected?

Hint: use (1) to compute the error in the energy norm (be careful with the squares). For the first term in the right-hand side of (1), you can use a composite quadrature. For the second term in the right-hand side of (1), you can use the formula given in (2). 

Hint: (1) only holds if the load vector is computed exactly. When you approximate the integrals in the load vector, use a very accurate quadrature. 


### Code Snippets

In [ ]:
def AssembleStiffnessMatrix1D(x):
    # Number of intervals
    N = x.size-1
    # 1) Allocate and initiate matrix
    A = ...

    # 2) Compute volume contributions by iterating over 
    #    intervals I_0 to I_(N-1):
    for i in range(0,N):
        # Mesh  size
        h = ...
        # Compute local stiffness matrix
        A_loc = ...*np.array([[1, -1],[-1, 1]])
        # Write local matrix into global
        A[i, i] += A_loc[0, 0]
        # Add three more matrix entries
        ...
        ...
        ...
    
    # 3) Compute natural boundary contributions
    # Add Robin on the right
    A[N, N] += ...
    
    return A

In [ ]:
def AssembleLoadVector1D(x, f, g_N, g_R):
    # Number of intervals
    N = ...
    # 1) Allocate and initiate global load vector
    b = ...
    # 2) Compute volume contributions by iterating over intervals 
    #    I_0 to I_(N-1):
    for i in range(0,N):
        # Use the code from the AssembleLoadVector1D used in the L2-projection
        ...

    # 3) Incorporate boundary values e.g
    # Neumann
    b[0] += g_N(0)

    # Robin
    b[N] += ...

    return b